In [1]:
%pip install osmium folium

Note: you may need to restart the kernel to use updated packages.


In [2]:
import vrp
import geographs

In [3]:
Depot_coord = (18.452521, -69.921330)
qty_clients = 30
cap_trucks = 80

In [4]:
print("Loading data... ")

geoGraph = geographs.GeoGraph(limit_coords=Depot_coord)
print("Data loaded! ")

Loading data... 
Available nodes before condition: 133161
Available nodes after condition: 12703
Data loaded! 


In [5]:
Depot_node, clients, qty_trucks = vrp.create_nodes(geoGraph, Depot_coord, qty_clients, cap_trucks)

print("Depot node: ", Depot_node)
print("qty of trucks: ", qty_trucks)

Depot node:  GeoNode 309456309 at pos (-699216191, 184523209), coords (18.4523209, -69.9216191)
qty of trucks:  4


In [6]:
qty_poblacion = 200
n_elite = 10
n_generations = 50
prob_de_mut = 0.5

solution = vrp.create_clusters(geoGraph, qty_clients, qty_trucks, cap_trucks, qty_poblacion, n_elite, n_generations, clients, prob_de_mut)

print("solution: ", solution)

solution:  [1, 0, 3, 1, 0, 2, 3, 1, 3, 0, 3, 2, 2, 0, 2, 0, 0, 2, 1, 0, 1, 2, 2, 0, 3, 2, 1, 1, 1, 3]


In [7]:
n_nearest_nodes = 3

routes, paths = vrp.create_routes(geoGraph, Depot_node, clients, qty_trucks, solution, n_nearest_nodes)

print("routes: ", routes)

for graph in routes:
    print(graph.get_nodes())


Truck 1/4
Len del client cluster: 8
Nearest nodes for client 5129295718: [308744088, 5956192082, 308747355]
Nearest nodes for client 308747355: [308744088, 5129295718, 5956192082]
Nearest nodes for client 310265651: [309774813, 6369049910, 5129295718]
Nearest nodes for client 308903663: [6369049910, 5956192082, 5129295718]
Nearest nodes for client 309774813: [310265651, 5129295718, 6369049910]
Nearest nodes for client 308744088: [308747355, 5956192082, 5129295718]
Nearest nodes for client 6369049910: [308903663, 5956192082, 5129295718]
Nearest nodes for client 5956192082: [308903663, 308744088, 6369049910]
Truck 2/4
Len del client cluster: 8
Nearest nodes for client 5959804791: [310419146, 310674372, 309173071]
Nearest nodes for client 309173071: [4134118767, 310674372, 5959804791]
Nearest nodes for client 7221003394: [4586126151, 310419146, 309674119]
Nearest nodes for client 309674119: [4586126151, 7221003394, 310419146]
Nearest nodes for client 310419146: [5959804791, 7221003394, 31

In [8]:
import folium
colors = ['red', 'blue', 'green', 'purple']

map = folium.Map(location=Depot_coord, zoom_start=17)

i = 0
for route in paths:
    for initial_node_name in route:
        for goal_node_name in route[initial_node_name]:
            
            initial_node = routes[i].get_node_by_name(initial_node_name)
            goal_node = routes[i].get_node_by_name(goal_node_name)

            initial_marker = [initial_node.lat, initial_node.lon]
            folium.Marker(initial_marker,popup=initial_node.name, icon=folium.Icon(color=colors[i])).add_to(map)
            
            goal_marker = [goal_node.lat, goal_node.lon]
            folium.Marker(goal_marker,popup=goal_node.name, icon=folium.Icon(color=colors[i])).add_to(map)

            path = route[initial_node_name][goal_node_name]
            points = []

            for node in path:
                point = [node.lat, node.lon]
                points.append(point)

            folium.PolyLine(points, color=colors[i], weight=5, opacity=1).add_to(map)
    i+=1

map